In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()

True

In [5]:
client = OpenAI()

In [25]:
dm_template = [
    {
        "role": "system",
        "content": """
        You are the DUNGEON MASTER agent crafting a story with {setting}. You will do one of many things:
        1. You will always output JSON according to provided json_schema.
        2. If prompted 'LOCKED ROOM' you will append a short description of a locked door situated in the {direction} to the following room details: {description}
        3. If prompted 'HIDDEN ITEM' you will output the short description of an {item} and append a subtle hint to the following room details: {description}
        4. If prompted 'INVENTORY' you will output the player's inventory in a short, descriptive manner. Here is the current inventory: {inventory}
        REQUIREMENTS:
        - The JSON must include an updated_description if 'LOCKED ROOM' or 'HIDDEN ITEM' are prompted else None.
        - The JSON must have an item_description if 'HIDDEN ITEM' is prompted else None.
        - The JSON must have an inventory_description if 'INVENTORY' is prompted else None.
        - The response in updated_description and item_description cannot resemble previous responses.
        - The response in inventory_description must resemble previous responses.
        - The json_schema must have the following properties: updated_description, item_description, inventory_description.
        """
    },
    {
        "role": "user",
        "content": "{prompt}"
    }
]

dm_config = [
    {
        "role": "system",
        "content": None
    },
    {
        "role": "user",
        "content": None
    }
]

def dm(prompt:str, setting:str, direction:str, description:str, item:str=None, inventory:list=None):

    # DOCSTRING
    """An agent that will update a room description based on a prompt.

    Args:
        prompt (str): Either 'LOCKED ROOM', 'HIDDEN ITEM', or 'INVENTORY'.
        setting (str): The setting of the room; what kind of location it is. Passed from the settings dictionary in modeuls.ai.config.
        direction (str): The direction of the locked door in the room.
        description (str): The description of the room.
        item (str, optional): The item that is hidden in the room. Defaults to None.
        inventory (str, optional): The player's inventory. Defaults to None.
    """

    global dm_config

    dm_config[0]['content'] = dm_template[0]['content'].format(setting=setting, direction=direction, description=description, item=item, inventory=inventory)
    dm_config[-1]['content'] = dm_template[-1]['content'].format(prompt=prompt)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=dm_config,
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "dm_output",
                "description": "The output of the dungeon master.",
                "schema": {
                    "type": "object",
                    "properties": {
                        "updated_description": {
                            "description": "The updated description of the room if prompted 'LOCKED ROOM' or 'HIDDEN ITEM'. None if not prompted.",
                            "type": ["string", "null"]
                        },
                        "item_description": {
                            "description": "The description of the item if prompted 'HIDDEN ITEM'. None if not prompted or is None.",
                            "type": ["string", "null"]
                        },
                        "inventory_description": {
                            "description": "The player's inventory if prompted 'INVENTORY'. None if not prompted or is None.",
                            "type": ["string", "null"]
                        },
                        "additionalProperties": False
                    }
                }
            }
        }
    )

    dm_config.insert(1, {"role": "system", "content": "PREVIOUS OUTPUT: " + f"{response.choices[0].message.content}"})
    
    return response.choices[0].message.content

In [30]:
response = json.loads(dm("inventory", "a cave", "north", "A dark, damp cave with a single torch on the wall. The faint rustling of leaves hints at something intriguing lying beneath.", 'adorned key', ['sword, shield, potion']))

In [31]:
response

{'updated_description': None,
 'item_description': None,
 'inventory_description': 'You currently possess a shining sword forged for battle, a sturdy shield to protect against enemies, and a potion that gleams with the promise of healing.'}